In [27]:
from loguru import logger
import ndf.download
from ndf.datamining import datamining
import warnings
import sys
import pandas as pd

warnings.simplefilter(action='ignore', category=FutureWarning)

log_level = 'ERROR'
logger.add(sys.stderr, format="{time} {level} {message}", filter="my_module", level=log_level)
# logger.add(sys.stdout, colorize=True, format="<green>{time}</green> <level>{message}</level>")
logger.add("ndfdownload.log", rotation="30 MB", level=log_level)

8

In [12]:
import requests
import time

MAX_RETRIES = 4
TIME_TO_WAIT = 10

def download_manager(url):
    r = None
    # logger.info(f'Download settings: MAX_RETRIES:{MAX_RETRIES} | TIME_TO_WAIT:{TIME_TO_WAIT}')
    for _ in range(MAX_RETRIES):
        try:
            r = requests.get(url, verify=False)
        except requests.exceptions.RequestException as e:
            print(f'Download fail, {e}')
        if r:
            print(f'Download Ok')
            break
        else:
            wait = TIME_TO_WAIT * (_ + 1) if _ < 3 else TIME_TO_WAIT * (_ * 3)
            print(f'Download fail, retry in {wait} seconds...')
            time.sleep(wait)
    return r

In [13]:
download_manager('https://www.tullettprebon.com/swap-execution-facility/daily-activity-summary.aspxa')

Download fail, ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Download fail, retry in 10 seconds...
Download fail, ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))
Download fail, retry in 20 seconds...


KeyboardInterrupt: 

In [11]:
from datetime import datetime

a = datetime.now()
a.date()

datetime.date(2023, 5, 29)

In [12]:
logger.remove()

In [13]:
d = ndf.download.download()
d.download_all()

bgc ok
tradition ok
prebontullet ok
gfi ok


In [14]:
mining = datamining()

In [15]:
mining.tulletprebon_calcs()

,Total for human,Volume
Class,,
BMF1,3.75B,3752800000
BMF2,874M,874000000
BMF3,470M,470000000
BROKEN,1.26B,1258000000
GT 1 YEAR,25M,25000000
LONGER,710M,710000000
PTAX,356.5M,356500000
TOMPTAX,20M,20000000
TOTAL,7.69B,7691550000


In [16]:
mining.bgc_calcs()

,Total for human,Volume
Class,,
BMF,1.3B,1295000000
BMF2,372.5M,372500000
BMF4,50M,50000000
BROKEN,346.5M,346500000
TOTAL,2.06B,2064000000


In [17]:
mining.tradition_calcs()

,Total for human,Volume
Class,,
BROKEN,52M,52000000
TOTAL,52M,52000000


In [18]:
mining.gfi_calcs()

,Total for human,Volume
Class,,
BMF,1.29B,1293000000
BMF2,725M,725000000
BROKEN,570M,570000000
TOTAL,2.59B,2588000000


In [20]:
tulletprebon['source'] = 'TulletPrebon'
tradition['source'] = 'Tradition'
bgc['source'] = 'bgc'
gfi['source'] = 'gfi'

In [24]:
bgc

,Total for human,Volume,source
Class,,,
BMF,1.3B,1295000000,bgc
BMF2,372.5M,372500000,bgc
BMF4,50M,50000000,bgc
BROKEN,346.5M,346500000,bgc
TOTAL,2.06B,2064000000,bgc


In [30]:
df_result = pd.concat([tulletprebon, tradition, bgc, gfi])
df_result

,Total for human,Volume,source
Class,,,
BMF1,3.75B,3752800000,TulletPrebon
BMF2,874M,874000000,TulletPrebon
BMF3,470M,470000000,TulletPrebon
BROKEN,1.26B,1258000000,TulletPrebon
GT 1 YEAR,25M,25000000,TulletPrebon
LONGER,710M,710000000,TulletPrebon
PTAX,356.5M,356500000,TulletPrebon
TOMPTAX,20M,20000000,TulletPrebon
TOTAL,7.69B,7691550000,TulletPrebon


In [44]:
df_totals = df_result.query('Class == "TOTAL"')
df_totals

,Total for human,Volume,source
Class,,,
TOTAL,7.69B,7691550000,TulletPrebon
TOTAL,52M,52000000,Tradition
TOTAL,2.06B,2064000000,bgc
TOTAL,2.59B,2588000000,gfi


In [48]:
df_totals.groupby(['Class']).sum()

,Volume
Class,
TOTAL,12395550000


In [57]:
from numerize import numerize

market_total = df_totals.sum()['Volume']
market_total = numerize.numerize(float(market_total))
market_total

'12.4B'